In [43]:
import pandas as pd
import numpy as np
import os
import requests
import env
import john_acquire as a

In [48]:
%load_ext autoreload
%autoreload 2

# Set the option to display all columns
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2023'
max_req = 100  # Specify the maximum number of observations to retrieve


In [61]:
inspections_df = a.get_health_inspection_data(year_to_retrieve, app_token)
inspections_df.head(5)

CSV file for 2023 already exists. Loading data from the CSV.


,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,50124301,YUMMY JUICE BAR,Bronx,737A,LYDIG AVENUE,10462.0,3472936151,"Juice, Smoothies, Fruit Salads",2023-01-03T00:00:00.000,Violations were cited in the following area(s).,10B,Anti-siphonage or back-flow prevention device ...,Not Critical,12.0,A,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Pre-permit (Operational) / Re-inspection,40.854471,-73.866174,211.0,13.0,22403.0,2049410.0,2.043190e+09,BX49
1,50106430,PAN TODO RICO,Queens,7617,ROOSEVELT AVE,11372.0,6466393116,Bakery Products/Desserts,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,32.0,C,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.747064,-73.889337,403.0,25.0,28700.0,4029880.0,4.012870e+09,QN28
2,50129821,MEI JUNG MEI CHINESE RESTAURANT,Brooklyn,1402,FLATBUSH AVENUE,11210.0,9292509943,Chinese,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,18.0,B,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Pre-permit (Operational) / Re-inspection,40.636380,-73.951435,314.0,45.0,77000.0,3120854.0,3.052260e+09,BK42
3,50057824,SERAFINA LUDLOW,Manhattan,98,RIVINGTON STREET,10002.0,2123589800,Italian,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,19.0,N,NaN,2023-10-24T06:00:07.000,Cycle Inspection / Initial Inspection,40.720111,-73.988463,103.0,1.0,3001.0,1084639.0,1.004110e+09,MN27
4,41399360,NEW RONG HANG RESTAURANT,Manhattan,38,ELDRIDGE STREET,10002.0,2126258999,Chinese,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04M,Live roaches in facility's food or non-food area.,Critical,27.0,B,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.715711,-73.993204,103.0,1.0,1600.0,1003876.0,1.003000e+09,MN27


In [53]:
inspections_df.head(5).to_csv('inspections_df.csv', index=False)

In [47]:
inspections_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73512 entries, 0 to 73511
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   camis                  73512 non-null  int64  
 1   dba                    73508 non-null  object 
 2   boro                   73512 non-null  object 
 3   building               73454 non-null  object 
 4   street                 73512 non-null  object 
 5   zipcode                72777 non-null  float64
 6   phone                  73512 non-null  object 
 7   cuisine_description    73512 non-null  object 
 8   inspection_date        73512 non-null  object 
 9   action                 73512 non-null  object 
 10  violation_code         73127 non-null  object 
 11  violation_description  73127 non-null  object 
 12  critical_flag          73512 non-null  object 
 13  score                  70147 non-null  float64
 14  grade                  41234 non-null  object 
 15  gr

In [3]:
g_places_api_key = env.g_places_api_key

In [77]:
import json
import requests

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Select the 5th row (index 4) for testing
row = inspections_df.iloc[4]

# Construct the text query using dba, street, and zipcode
text_query = f"{row['dba']} {row['street']}"
if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
    text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query

# Define the query
data = {
    'textQuery': text_query
}



In [ ]:
# Make the POST request
response = requests.post(url, headers=headers, json=data)

# Process the response
if response.status_code == 200:
    data = response.json().get('places', [])
    print("Received data:", data)
else:
    print(f"Failed to retrieve data for {text_query}: {response.status_code}")


In [79]:
data

[{'id': 'ChIJGXeLPyhawokRXK9zCnvTMuQ',
  'formattedAddress': '38 Eldridge St, New York, NY 10002, USA',
  'displayName': {'text': 'Rong Hang', 'languageCode': 'en'},
  'reviews': [{'name': 'places/ChIJGXeLPyhawokRXK9zCnvTMuQ/reviews/ChdDSUhNMG9nS0VJQ0FnSUNoMllXb2h3RRAB',
    'relativePublishTimeDescription': '8 months ago',
    'rating': 4,
    'text': {'text': "It's an authentic Fu Zhou restaurant but can be a little daunting for people not from this area. Servers spoke Fu Zhou accents and menu items mostly are at market price. However the seafood is very fresh and tasty. You really have to bring a friend who knows their language so that they can order off menu items!\n\nThey allow smoking in the restaurant which I don't really like. It can get very noisy too sometimes.",
     'languageCode': 'en'},
    'originalText': {'text': "It's an authentic Fu Zhou restaurant but can be a little daunting for people not from this area. Servers spoke Fu Zhou accents and menu items mostly are at ma

In [ ]:
import pandas as pd
import json
import requests

# Initialize an empty list to store the results
results = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iterrows():
    # Construct the text query using dba, street, and zipcode
    text_query = f"{row['dba']} {row['street']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the result along with the camis value to the results list
        for place in places_data:
            results.append({
                'camis': row['camis'],
                'place_id': place.get('id', ''),
                'display_name': place.get('displayName', ''),
                'formatted_address': place.get('formattedAddress', ''),
                'reviews': place.get('reviews', [])
            })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Display the first few rows of the results DataFrame
print(results_df.head())


In [82]:
import pandas as pd
import json
import requests

# Initialize empty lists to store the results for places and reviews
places_results = []
reviews_results = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iloc[2:3].iterrows():
    # Construct the text query using dba, street, and zipcode
    text_query = f"{row['dba']} {row['street']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the place details and reviews to the respective lists
        for place in places_data:
            places_results.append({
                'camis': row['camis'],
                'place_id': place.get('id', ''),
                'display_name': place.get('displayName', ''),
                'formatted_address': place.get('formattedAddress', '')
            })
            for review in place.get('reviews', []):
                reviews_results.append({
                    'camis': row['camis'],
                    'place_id': place.get('id', ''),
                    'review': review  # Assuming 'review' contains the review details
                })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")

# Convert the results lists to DataFrames
places_df = pd.DataFrame(places_results)
reviews_df = pd.DataFrame(reviews_results)


In [88]:
import pandas as pd
import json
import requests

# Initialize empty lists to store the results for places and reviews
places_results = []
reviews_results = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iloc[2:3].iterrows():
    # Construct the text query using dba, street, and zipcode
    text_query = f"{row['dba']} {row['street']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the place details and reviews to the respective lists
        for place in places_data:
            places_results.append({
                'camis': row['camis'],
                'place_id': place.get('id', ''),
                'display_name': place.get('displayName', {}).get('text', ''),
                'formatted_address': place.get('formattedAddress', '')
            })
            for review in place.get('reviews', []):
                reviews_results.append({
                    'camis': row['camis'],
                    'place_id': place.get('id', ''),
                    'review_name': review.get('name', ''),
                    'review_relative_time': review.get('relativePublishTimeDescription', ''),
                    'review_rating': review.get('rating', ''),
                    'review_text': review.get('text', {}).get('text', ''),
                    'review_language': review.get('text', {}).get('languageCode', ''),
                    'author_display_name': review.get('authorAttribution', {}).get('displayName', ''),
                    'author_uri': review.get('authorAttribution', {}).get('uri', ''),
                    'author_photo_uri': review.get('authorAttribution', {}).get('photoUri', ''),
                    'publish_time': review.get('publishTime', '')
                })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")

# Convert the results lists to DataFrames
places_df = pd.DataFrame(places_results)
reviews_df = pd.DataFrame(reviews_results)

In [92]:
inspections_df.iloc[2:3]

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
2,50129821,MEI JUNG MEI CHINESE RESTAURANT,Brooklyn,1402,FLATBUSH AVENUE,11210.0,9292509943,Chinese,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,18.0,B,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Pre-permit (Operational) / Re-inspection,40.63638,-73.951435,314.0,45.0,77000.0,3120854.0,3.052260e+09,BK42


In [89]:
places_df.head()

,camis,place_id,display_name,formatted_address
0,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,Goop li asian love food,"1402 Flatbush Ave, Brooklyn, NY 11210, USA"


In [90]:
places_df.display_name[0]

'Goop li asian love food'

In [91]:
reviews_df.head()

,camis,place_id,review_name,review_relative_time,review_rating,review_text,review_language,author_display_name,author_uri,author_photo_uri,publish_time
0,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,places/ChIJVQB8JUxbwokRB2EYAqesQvE/reviews/Chd...,a year ago,4,I used to go there all the time. Used to get d...,en,alwaysarcastic**,https://www.google.com/maps/contrib/1017042546...,https://lh3.googleusercontent.com/a-/ALV-UjWrF...,2022-10-20T10:55:46Z
1,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,places/ChIJVQB8JUxbwokRB2EYAqesQvE/reviews/Chd...,4 months ago,4,A lot of love for them. They are good on most ...,en,James Perkins,https://www.google.com/maps/contrib/1172583623...,https://lh3.googleusercontent.com/a-/ALV-UjXLp...,2023-06-19T13:10:41Z
2,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,places/ChIJVQB8JUxbwokRB2EYAqesQvE/reviews/Chd...,a year ago,3,This fried rice is always always always over c...,en,T J,https://www.google.com/maps/contrib/1121607201...,https://lh3.googleusercontent.com/a-/ALV-UjWv7...,2022-06-13T16:17:06Z
3,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,places/ChIJVQB8JUxbwokRB2EYAqesQvE/reviews/Chd...,10 months ago,5,Best fries and green plantain with wings they ...,en,Melissa,https://www.google.com/maps/contrib/1175674079...,https://lh3.googleusercontent.com/a-/ALV-UjWbf...,2022-12-06T20:40:59Z
4,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,places/ChIJVQB8JUxbwokRB2EYAqesQvE/reviews/ChZ...,3 years ago,5,One of the best places for Chinese food in the...,en,Asya Stepnova,https://www.google.com/maps/contrib/1102620964...,https://lh3.googleusercontent.com/a-/ALV-UjWgg...,2020-08-24T01:10:36Z


In [99]:
reviews_df.author_uri[0]

'https://www.google.com/maps/contrib/101704254655317829682/reviews'

In [ ]:

# Display the first few rows of the Places and Reviews DataFrames
print("Places DataFrame:")
print(places_df.head())
print("\nReviews DataFrame:")
print(reviews_df.head())


In [105]:
import pandas as pd
import json
import requests

# Initialize empty lists to store the results for places and reviews
places_results = []
reviews_results = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iloc[2:3].iterrows():
    # Construct the text query using dba, street, and zipcode
    text_query = f"{row['dba']} {row['street']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the place details and reviews to the respective lists
        for place in places_data:
            places_results.append({
                'camis': row['camis'],
                'place_id': place.get('id', ''),
                'display_name': place.get('displayName', {}).get('text', ''),
                'formatted_address': place.get('formattedAddress', '')
            })
            for review in place.get('reviews', []):
                # Extract the unique review identifier
                review_id = review.get('name', '').split('/')[-1]
                # Extract the unique contributor ID from the author_uri
                contributor_id = review.get('authorAttribution', {}).get('uri', '').split('/')[-2]
                reviews_results.append({
                    'camis': row['camis'],
                    'place_id': place.get('id', ''),
                    'review_id': review_id,
                    # 'review_relative_time': review.get('relativePublishTimeDescription', ''),
                    'review_rating': review.get('rating', ''),
                    'review_text': review.get('text', {}).get('text', ''),
                    'review_language': review.get('text', {}).get('languageCode', ''),
                    # 'author_display_name': review.get('authorAttribution', {}).get('displayName', ''),
                    'contributor_id': contributor_id,
                    # 'author_photo_uri': review.get('authorAttribution', {}).get('photoUri', ''),
                    'publish_time': review.get('publishTime', '')
                })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")

# Convert the results lists to DataFrames
places_df = pd.DataFrame(places_results)
reviews_df = pd.DataFrame(reviews_results)

In [109]:
data

{'textQuery': 'MEI JUNG MEI CHINESE RESTAURANT FLATBUSH AVENUE 11210'}

In [106]:
places_df.head()

,camis,place_id,display_name,formatted_address
0,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,Goop li asian love food,"1402 Flatbush Ave, Brooklyn, NY 11210, USA"


In [107]:
places_df.display_name[0]

'Goop li asian love food'

In [108]:
reviews_df.head()

,camis,place_id,review_id,review_rating,review_text,review_language,contributor_id,publish_time
0,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,ChdDSUhNMG9nS0VJQ0FnSURlMjZXOGpBRRAB,4,I used to go there all the time. Used to get d...,en,101704254655317829682,2022-10-20T10:55:46Z
1,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,ChdDSUhNMG9nS0VJQ0FnSUNKOE56TTR3RRAB,4,A lot of love for them. They are good on most ...,en,117258362311722126898,2023-06-19T13:10:41Z
2,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,ChdDSUhNMG9nS0VJQ0FnSUR5a3RPVjN3RRAB,3,This fried rice is always always always over c...,en,112160720159424315379,2022-06-13T16:17:06Z
3,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,ChdDSUhNMG9nS0VJQ0FnSUQtN2ZyVXZ3RRAB,5,Best fries and green plantain with wings they ...,en,117567407903896857593,2022-12-06T20:40:59Z
4,50129821,ChIJVQB8JUxbwokRB2EYAqesQvE,ChZDSUhNMG9nS0VJQ0FnSUNDM3NMdFlnEAE,5,One of the best places for Chinese food in the...,en,110262096475130641021,2020-08-24T01:10:36Z


In [123]:
import pandas as pd
import json
import requests

# Initialize empty lists to store the results for places, reviews, and API logs
places_results = []
reviews_results = []
api_logs = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Parameters for saving progress
save_interval = 10  # Save after every 10 rows
current_row = 0

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iloc[39:40].iterrows():
    current_row += 1
    # Construct the text query using dba, building, street, borough, and zipcode
    text_query = f"{row['dba']} {row['building']} {row['street']} {row['boro']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query
    if pd.notnull(row['phone']):  # Check if the phone number is not null
        text_query += f" {row['phone']}"  # Append the phone number to the query
    if pd.notnull(row['cuisine_description']):  # Check if the cuisine type is not null
        text_query += f" {row['cuisine_description']}"  # Append the cuisine type to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the place details and reviews to the respective lists
        for place in places_data:
            display_name = place.get('displayName', {}).get('text', '')
            formatted_address = place.get('formattedAddress', '')
            places_results.append({
                'camis': row['camis'],
                'place_id': place.get('id', ''),
                'display_name': display_name,
                'formatted_address': formatted_address
            })
            for review in place.get('reviews', []):
                # Extract the unique review identifier and contributor ID
                review_id = review.get('name', '').split('/')[-1]
                contributor_id = review.get('authorAttribution', {}).get('uri', '').split('/')[-2]
                reviews_results.append({
                    'camis': row['camis'],
                    'place_id': place.get('id', ''),
                    'review_id': review_id,
                    'review_relative_time': review.get('relativePublishTimeDescription', ''),
                    'review_rating': review.get('rating', ''),
                    'review_text': review.get('text', {}).get('text', ''),
                    'review_language': review.get('text', {}).get('languageCode', ''),
                    'author_display_name': review.get('authorAttribution', {}).get('displayName', ''),
                    'contributor_id': contributor_id,
                    'author_photo_uri': review.get('authorAttribution', {}).get('photoUri', ''),
                    'publish_time': review.get('publishTime', '')
                })
            # Log the API call with display_name and formatted_address
            api_logs.append({
                'row_number': current_row,
                'query': text_query,
                'status_code': response.status_code,
                'display_name': display_name,
                'formatted_address': formatted_address
            })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")
        api_logs.append({
            'row_number': current_row,
            'query': text_query,
            'status_code': response.status_code,
            'display_name': '',
            'formatted_address': ''
        })

    # Save progress at regular intervals
    if current_row % save_interval == 0:
        pd.DataFrame(places_results).to_csv('places_progress.csv', index=False)
        pd.DataFrame(reviews_results).to_csv('reviews_progress.csv', index=False)
        pd.DataFrame(api_logs).to_csv('api_log_progress.csv', index=False)

# Save the final results
pd.DataFrame(places_results).to_csv('places_final.csv', index=False)
pd.DataFrame(reviews_results).to_csv('reviews_final.csv', index=False)
pd.DataFrame(api_logs).to_csv('api_log_final.csv', index=False)


In [117]:
inspections_df.iloc[38:42]

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
38,50099824,INDIA HOUSE,Brooklyn,586,NOSTRAND AVENUE,11216.0,7188577011,Indian,2023-01-03T00:00:00.000,Establishment Closed by DOHMH. Violations were...,06E,"Sanitized equipment or utensil, including in-u...",Critical,69.0,NaN,NaN,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.678687,-73.949677,303.0,36.0,24700.0,3053936.0,3.018660e+09,BK61
39,50106021,TAMMAM CAFE,Brooklyn,735,NOSTRAND AVENUE,11216.0,3477894734,Coffee/Tea,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04A,Food Protection Certificate (FPC) not held by ...,Critical,10.0,A,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.672147,-73.950277,308.0,35.0,31702.0,3031915.0,3.012480e+09,BK61
40,50003774,GOLDEN KRUST CARIBBEAN BAKERY & GRILL,Brooklyn,4108,FARRAGUT ROAD,11210.0,7185766021,Caribbean,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,14.0,NaN,NaN,2023-10-24T06:00:07.000,Cycle Inspection / Initial Inspection,40.636919,-73.937311,317.0,45.0,83600.0,3114504.0,3.050160e+09,BK91
41,50076302,PAPA JOHN'S,Bronx,2241,WESTCHESTER AVENUE,10462.0,7185977272,Pizza,2023-01-03T00:00:00.000,Establishment Closed by DOHMH. Violations were...,06D,"Food contact surface not properly washed, rins...",Critical,56.0,NaN,NaN,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.834317,-73.850571,210.0,18.0,20601.0,2041407.0,2.039630e+09,BX59


In [124]:
response.json().get('places', [])

[{'id': 'ChIJy4EKcZ1bwokRAxycHPSTeSU',
  'formattedAddress': '735 Nostrand Ave., Brooklyn, NY 11216, USA',
  'displayName': {'text': '735 Nostrand Ave.'}}]

In [125]:
import pandas as pd
import json
import requests

# Initialize empty lists to store the results for places, reviews, and API logs
places_results = []
reviews_results = []
api_logs = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Parameters for saving progress
save_interval = 10  # Save after every 10 rows
current_row = 0

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iloc[39:40].iterrows():
    current_row += 1
    # Construct the text query using dba, building, street, borough, and zipcode
    text_query = f"{row['dba']} {row['building']} {row['street']} {row['boro']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query
    if pd.notnull(row['phone']):  # Check if the phone number is not null
        text_query += f" {row['phone']}"  # Append the phone number to the query
    if pd.notnull(row['cuisine_description']):  # Check if the cuisine type is not null
        text_query += f" {row['cuisine_description']}"  # Append the cuisine type to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the place details and reviews to the respective lists
        for place in places_data:
            display_name = place.get('displayName', {}).get('text', '').strip()
            formatted_address = place.get('formattedAddress', '').strip()
            address_first_part = formatted_address.split(',')[0].strip()

            # Check if the display name matches the first part of the formatted address
            if display_name.lower() == address_first_part.lower():
                note = 'Possible address only, no specific business match'
            else:
                note = ''
                places_results.append({
                    'camis': row['camis'],
                    'place_id': place.get('id', ''),
                    'display_name': display_name,
                    'formatted_address': formatted_address
                })

                for review in place.get('reviews', []):
                    # Extract the unique review identifier and contributor ID
                    review_id = review.get('name', '').split('/')[-1]
                    contributor_id = review.get('authorAttribution', {}).get('uri', '').split('/')[-2]
                    reviews_results.append({
                        'camis': row['camis'],
                        'place_id': place.get('id', ''),
                        'review_id': review_id,
                        'review_relative_time': review.get('relativePublishTimeDescription', ''),
                        'review_rating': review.get('rating', ''),
                        'review_text': review.get('text', {}).get('text', ''),
                        'review_language': review.get('text', {}).get('languageCode', ''),
                        'author_display_name': review.get('authorAttribution', {}).get('displayName', ''),
                        'contributor_id': contributor_id,
                        'author_photo_uri': review.get('authorAttribution', {}).get('photoUri', ''),
                        'publish_time': review.get('publishTime', '')
                    })

            # Log the API call with display_name, formatted_address, and note
            api_logs.append({
                'row_number': current_row,
                'query': text_query,
                'status_code': response.status_code,
                'display_name': display_name,
                'formatted_address': formatted_address,
                'note': note
            })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")
        api_logs.append({
            'row_number': current_row,
            'query': text_query,
            'status_code': response.status_code,
            'display_name': '',
            'formatted_address': '',
            'note': 'API call failed'
        })

    # Save progress at regular intervals
    if current_row % save_interval == 0:
        pd.DataFrame(places_results).to_csv('places_progress.csv', index=False)
        pd.DataFrame(reviews_results).to_csv('reviews_progress.csv', index=False)
        pd.DataFrame(api_logs).to_csv('api_log_progress.csv', index=False)

# Save the final results
pd.DataFrame(places_results).to_csv('places_final.csv', index=False)
pd.DataFrame(reviews_results).to_csv('reviews_final.csv', index=False)
pd.DataFrame(api_logs).to_csv('api_log_final.csv', index=False)
